In [2]:
# First, install the missing package
!pip install pydub


In [3]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [5]:
import librosa
import soundfile as sf
import os
import numpy as np
from pydub import AudioSegment

In [6]:
input_root = "/Users/sajin/dataset/"

output_root = "wav_dataset/"
os.makedirs(output_root,exist_ok=True)

for folder in ["C_major","Not_C_major"]:
    inp = os.path.join(input_root,folder)
    out = os.path.join(output_root,folder)
    os.makedirs(out,exist_ok = True)

    for file in os.listdir(inp):
        if file.lower().endswith(".m4a"):
            in_path = os.path.join(inp,file)
            out_path = os.path.join(out,file.rsplit('.',1)[0]+".wav")
            audio = AudioSegment.from_file(in_path,format = "m4a")
            audio = audio.set_frame_rate(16000).set_channels(1)
            audio.export(out_path, format="wav")
            print("converted:",file)

converted: C_3_EGC_M.m4a
converted: C_4_CEG_S.m4a
converted: C_3_GCE_H.m4a
converted: C_3_GCE_M.m4a
converted: C_3_EGC_H.m4a
converted: C_4_GCE_M.m4a
converted: C_5_CEG_S.m4a
converted: C_4_EGC_H.m4a
converted: C_3_CEG_S.m4a
converted: C_4_EGC_M.m4a
converted: C_4_GCE_H.m4a
converted: C_4_GCE_S.m4a
converted: C_5_CEG_M.m4a
converted: C_3_CEG_H.m4a
converted: C_3_CEG_M.m4a
converted: C_4_EGC_S.m4a
converted: C_5_CEG_H.m4a
converted: C_3_EGC_S.m4a
converted: C_4_CEG_M.m4a
converted: C_3_GCE_S.m4a
converted: C_4_CEG_H.m4a
converted: 8.m4a
converted: 9.m4a
converted: 11.m4a
converted: 10.m4a
converted: 12.m4a
converted: 13.m4a
converted: 17.m4a
converted: 16.m4a
converted: 14.m4a
converted: 15.m4a
converted: 18.m4a
converted: 19.m4a
converted: 21.m4a
converted: 20.m4a
converted: 1.m4a
converted: 2.m4a
converted: 3.m4a
converted: 7.m4a
converted: 6.m4a
converted: 4.m4a
converted: 5.m4a


In [8]:
def extract_chroma(path, sr=16000):
    y,sr = librosa.load(path,sr=sr,mono=True)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return np.mean(chroma,axis=1)

root = "wav_dataset/"
X,y,files = [], [], []
label_map = {"C_major":1,"Not_C_major":0}
for label_name,lbl in label_map.items():
    folder = os.path.join(root,label_name)
    for fname in os.listdir(folder):
        if fname.lower().endswith(".wav"):
            path = os.path.join(folder,fname)
            feat = extract_chroma(path)
            X.append(feat)
            y.append(lbl)
            files.append(path)
X = np.array(X)
y = np.array(y)
print("X shape:",X.shape,"y shape:",y.shape) 

X shape: (42, 12) y shape: (42,)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42,stratify = y)
model = RandomForestClassifier(n_estimators = 200,random_state = 42)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("Accuracy:",accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred,target_names=["Not C major","C major"]))
print("confusion matrix:\n",confusion_matrix(y_test,y_pred))

Accuracy: 0.8888888888888888
              precision    recall  f1-score   support

 Not C major       1.00      0.80      0.89         5
     C major       0.80      1.00      0.89         4

    accuracy                           0.89         9
   macro avg       0.90      0.90      0.89         9
weighted avg       0.91      0.89      0.89         9

confusion matrix:
 [[4 1]
 [0 4]]


In [11]:
import joblib
joblib.dump(model,"c_major_randomforest_model.joblib")

['c_major_randomforest_model.joblib']